In [2]:
import pandas as pd
import os
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import dash_bootstrap_components as dbc
import gdown
import zipfile

In [ ]:
file_id = "1k2b0EHMDGu-3s6f4kJcQ3e-phaR4LKlt"
output_file = "feature_store_full_data.zip"

url = f"https://drive.google.com/uc?id={file_id}"
gdown.download(url, output_file)

In [ ]:
zip_file_path = "./" + output_file  # Replace with the path of your zip file
data_folder_name = 'feature_store_d'
destination_folder = "./"+data_folder_name  # Replace with the destination folder where you want to extract the files

# Open the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all the contents of the zip file to the destination folder
    zip_ref.extractall(destination_folder)

In [3]:
subfolder_name = 'out'
data_folder_name = os.path.join(data_folder_name, subfolder_name)

dfs = []
for file in os.listdir(data_folder_name):
    dfs.append(pd.read_parquet(os.path.join(data_folder_name,file)))
    # print('out/' + file)

In [4]:
df = pd.concat(dfs)
print(df.shape)

(75302, 548)


In [5]:
for col in df.columns:
    if pd.api.types.is_integer_dtype(df[col]) and df[col].dtype == np.int32:
        df[col] = df[col].astype('int64')

In [6]:
df = df.dropna(axis='columns')

In [7]:
df.head

<bound method NDFrame.head of      CFK_CIF_NBR AZ_BATCH_DATE CFMR_BIRTHDATE CFMR_DATE_OPENED CFMR_DEATH_DT  \
0        7006335    2022-08-12     2022-09-05       2022-06-23    2022-03-27   
1        7006337    2022-08-12     2021-06-09       2021-05-12    2021-06-27   
2        7006338    2022-08-12     2021-10-15       2021-06-04    2021-09-25   
3        7006339    2022-08-12     2021-01-17       2021-04-07    2021-04-29   
4        7006340    2022-08-12     2022-01-03       2022-04-05    2022-03-20   
...          ...           ...            ...              ...           ...   
9027     8006317    2022-12-30     2021-11-23       2021-12-28    2022-02-27   
9028     8006320    2022-12-30     2021-07-21       2021-11-29    2021-07-22   
9029     8006325    2022-12-30     2022-10-18       2023-01-21    2022-09-20   
9030     8006326    2022-12-30     2020-10-07       2020-11-29    2020-11-07   
9031     8006327    2022-12-30     2022-07-16       2022-03-10    2022-04-14   

     CFMR

In [ ]:
# pip install -U kaleido


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Write down here the address where you want the pdf file to be saved

In [18]:
folder_path = 'feature_store_pdfs'

if not os.path.exists(folder_path):
        os.mkdir(folder_path)



In [9]:
for col in df.columns:
    if pd.api.types.is_integer_dtype(df[col]) and df[col].dtype == np.int32:
        df[col] = df[col].astype('int64')

In [10]:
Time = 'AZ_BATCH_DATE'

df[Time] = pd.to_datetime(df[Time])
df['year'] = df[Time].dt.year
df['quarter'] = df[Time].dt.quarter
df['month'] = df[Time].dt.month
df['week'] = df[Time].dt.isocalendar().week
df['day'] = df[Time].dt.day
df['hour'] = df[Time].dt.hour
df['day_of_week'] = df[Time].dt.dayofweek 

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Assuming df is your DataFrame
columns = df.select_dtypes(include=['float64', 'int64']).columns  # Selecting numeric columns
output_folder = folder_path # Specify your output folder here

for column in columns:
    # Create a PDF object with specific file name

    if not os.path.exists(os.path.join(output_folder,'Histogram')):
        os.mkdir(os.path.join(output_folder,'Histogram'))
        
    pdf_pages = PdfPages(f'{output_folder}/Histogram/Histogram, Mean, Min, and Max of {column}.pdf')

    plt.figure(figsize=(10, 6))

    # Create histogram for each column
    plt.hist(df[column].dropna(), bins=50, color='blue', alpha=0.5)
    mean = np.mean(df[column])
    min_value = np.min(df[column])
    max_value = np.max(df[column])

    # Add vertical lines for mean, min and max
    plt.axvline(mean, color='r', linestyle='dashed', linewidth=2)
    plt.axvline(min_value, color='g', linestyle='dashed', linewidth=2)
    plt.axvline(max_value, color='b', linestyle='dashed', linewidth=2)

    # Add text for mean, min and max
    plt.text(mean, plt.ylim()[1]*0.9, f'Mean: {mean:.2f}', color='r')
    plt.text(min_value, plt.ylim()[1]*0.8, f'Min: {min_value:.2f}', color='g')
    plt.text(max_value, plt.ylim()[1]*0.7, f'Max: {max_value:.2f}', color='b')

    plt.title(f'Histogram, Mean, Min, and Max of {column}', size=15)
    plt.xlabel(column, size=12)
    plt.ylabel('Frequency', size=12)

    # Save the figure to the PDF file
    pdf_pages.savefig(plt.gcf())

    # Close the figure to free up memory
    plt.close()

    # Save the PDF file to disk
    pdf_pages.close()


In [21]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
import numpy as np

# Assuming df is your DataFrame and Time is your datetime index
df_temp = df.copy()
df_temp.set_index(Time, inplace=True)
columns = df_temp.select_dtypes(include=[np.number]).columns  # select numeric columns
agg_funcs = ['std', 'mean', 'median']
output_folder = folder_path   # Specify your output folder here

for column in columns:
    # Create a PDF object with specific file name
    if not os.path.exists(os.path.join(output_folder,'Time_Series')):
        os.mkdir(os.path.join(output_folder,'Time_Series'))
    pdf_pages = PdfPages(f'{output_folder}/Time_Series/Time Series of {column} with Aggregation Functions.pdf')

    plt.figure(figsize=(10, 6))

    for agg_func in agg_funcs:
        # Calculate monthly aggregates
        monthly_data = df_temp[column].resample('M').agg(agg_func)

        # Plot the time series
        plt.plot(monthly_data.index, monthly_data, label=f'Monthly {agg_func} of {column}')

    plt.title(f'Time Series of {column} with Aggregation Functions', size=15)
    plt.xlabel('Time', size=12)
    plt.ylabel(column, size=12)
    plt.legend()

    # Save the figure to the PDF file
    pdf_pages.savefig(plt.gcf())

    # Close the figure to free up memory
    plt.close()

    # Save the PDF file to disk
    pdf_pages.close()



In [11]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.dates as mdates

# Assuming df is your DataFrame and 'Time' is your datetime column
df_temp = df.copy()
df_temp['Time'] = pd.to_datetime(df_temp[Time])  # Convert 'Time' to datetime
df_temp.set_index('Time', inplace=True)  # Set 'Time' as index

# Add new columns for Year and Day of Week
df_temp['Year'] = df_temp.index.year
df_temp['Day_of_Week'] = df_temp.index.dayofweek

# Get numeric columns
numeric_columns = df_temp.select_dtypes(include=[np.number]).columns.tolist()

# Get unique years
years = df_temp['Year'].unique().tolist()
years.append('ALL')

output_folder = folder_path  # Specify your output folder here

for year in years:
    for column in numeric_columns:
        # Create a PDF object with specific file name
        if not os.path.exists(os.path.join(output_folder,'Yearly_Trend')):
            os.mkdir(os.path.join(output_folder,'Yearly_Trend'))
            
        pdf_pages = PdfPages(f'{output_folder}/Yearly_Trend/Yearly_Trend_{column}_{year}.pdf')

        if year == 'ALL':
            data = df_temp.copy()
        else:
            data = df_temp[df_temp['Year'] == year]

        monthly_avg = data[column].resample('M').mean()

        fig, ax = plt.subplots(figsize=(10, 6))
        ax.plot(monthly_avg.index, monthly_avg, label='Yearly Trend of {} in {}'.format(column, year))
        ax.set_title(f'Yearly Trend of {column} in {year}')
        ax.set_xlabel('Time')
        ax.set_ylabel(column)
        ax.legend()

        # Save the figure to the PDF file
        pdf_pages.savefig(fig)

        # Close the figure to free up memory
        plt.close(fig)

        # Save the PDF file to disk
        pdf_pages.close()


In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.dates as mdates

# Assuming df is your DataFrame and Time is your datetime index
df_temp = df.copy()
df_temp.set_index(Time, inplace=True)
columns = df_temp.select_dtypes(include=[np.number]).columns  # select numeric columns
output_folder = folder_path   # Specify your output folder here

for column in columns:
    # Create a PDF object with specific file name
    if not os.path.exists(os.path.join(output_folder,'Monthly_Average')):
            os.mkdir(os.path.join(output_folder,'Monthly_Average'))

    pdf_pages = PdfPages(f'{output_folder}/Monthly_Average/Monthly Average of {column}.pdf')

    plt.figure(figsize=(10, 6))

    # Compute monthly average
    monthly_avg = df_temp[column].resample('M').mean()

    # Create line plot for monthly average
    plt.plot(monthly_avg.index, monthly_avg, label='Monthly Average of {}'.format(column))
    
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
    
    plt.title(f'Monthly Average of {column}', size=15)
    plt.xlabel('Time', size=12)
    plt.ylabel(column, size=12)
    plt.legend()

    # Save the figure to the PDF file
    pdf_pages.savefig(plt.gcf())

    # Close the figure to free up memory
    plt.close()

    # Save the PDF file to disk
    pdf_pages.close()


In [16]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
import numpy as np

# Assuming df is your DataFrame and Time is your datetime index
df_temp = df.copy()
df_temp = df_temp.dropna(subset=[Time])
df_temp.set_index(Time, inplace=True)
columns = df_temp.select_dtypes(include=[np.number]).columns  # select numeric columns
object_columns = df_temp.select_dtypes(include=['object']).columns  # select object columns
output_folder = folder_path
# Function to get filtered options for group columns
def get_filtered_options(df, object_columns):
    new_options = []
    for col in object_columns:
        if df[col].nunique() <= 15:
            new_options.append(col)
    return new_options

filtered_options = get_filtered_options(df_temp, object_columns)

# Loop through numeric columns
for column in columns:
    # Loop through group columns
    for group_column in filtered_options:
        groups = df_temp[group_column].unique()  # Unique values of group column

        # Create a PDF object with specific file name
        if not os.path.exists(os.path.join(output_folder,'Time_Series_Groups')):
            os.mkdir(os.path.join(output_folder,'Time_Series_Groups'))

        pdf_pages = PdfPages(f'{output_folder}/Time_Series_Groups/Time Series of {column} on Different {group_column} Groups.pdf')

        plt.figure(figsize=(10, 6))

        # Loop through each unique group
        for group in groups:

            # Compute monthly average for specific group
            group_data = df_temp[df_temp[group_column] == group][column]
            
            # Convert to numeric and drop NA
            group_data = pd.to_numeric(group_data, errors='coerce').dropna()

            monthly_avg = group_data.resample('M').mean().dropna()


            # Create line plot for monthly average
            plt.plot(monthly_avg.index, monthly_avg, label=f'{group_column}: {group}')

        plt.title(f'Monthly Average of {column} for Different {group_column} Groups', size=15)
        plt.xlabel('Time', size=12)
        plt.ylabel(column, size=12)
        plt.legend()

        # Save the figure to the PDF file
        pdf_pages.savefig(plt.gcf())

        # Close the figure to free up memory
        plt.close()

        # Save the PDF file to disk
        pdf_pages.close()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
from itertools import combinations

# Assuming df is your DataFrame and 'Time' is your datetime index
df_temp = df.copy()

# Get numeric columns
numeric_columns = df_temp.select_dtypes(include=[np.number]).columns.tolist()

# Generate all combinations of numeric columns
col_pairs = list(combinations(numeric_columns, 2))

output_folder = folder_path  # Specify your output folder here

for col1, col2 in col_pairs:
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=df_temp, x=col1, y=col2)
    plt.title(f'Correlation between {col1} and {col2}')
    correlation = df_temp[[col1, col2]].corr().iloc[0, 1]
    plt.text(0.1, 0.9, 'Correlation: {:.2f}'.format(correlation), transform=plt.gca().transAxes)

    if not os.path.exists(os.path.join(output_folder,'Correlation')):
            os.mkdir(os.path.join(output_folder,'Correlation'))

    pdf_pages = PdfPages(f'{output_folder}/Correlation/Correlation_{col1}_{col2}.pdf')
    pdf_pages.savefig(plt.gcf())
    plt.close()
    pdf_pages.close()


In [60]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Assuming 'Time' is your datetime column
df_temp = df.copy()
df_temp['Time'] = pd.to_datetime(df_temp[Time])  # Convert 'Time' to datetime
df_temp['Year'] = df_temp[Time].dt.year  # Add a new column for Year
df_temp['Day_of_Week'] = df_temp[Time].dt.dayofweek  # Add a new column for Day of Week

# Get numeric columns
numeric_columns = df_temp.select_dtypes(include=[np.number]).columns.tolist()

# Get unique years
years = df_temp['Year'].unique().tolist()
years.append('ALL')

output_folder = folder_path   # Specify your output folder here

for year in years:
    for column in numeric_columns:
        # Create a PDF object with specific file name
        if not os.path.exists(os.path.join(output_folder,'Average_Weekday')):
            os.mkdir(os.path.join(output_folder,'Average_Weekday'))

        pdf_pages = PdfPages(f'{output_folder}/Average_Weekday/Average_{column}_Weekday_{year}.pdf')

        if year == 'ALL':
            data = df_temp.copy()
        else:
            data = df_temp[df_temp['Year'] == year]

        weekday_avg = data.groupby('Day_of_Week')[column].mean()

        fig, ax = plt.subplots(figsize=(10, 6))
        ax.bar(weekday_avg.index, weekday_avg, label='Average of {}'.format(column))
        ax.set_title(f'Average of {column} for Each Day of the Week in {year}')
        ax.set_xlabel('Day of the Week')
        ax.set_ylabel('Average')
        ax.legend()

        # Save the figure to the PDF file
        pdf_pages.savefig(fig)

        # Close the figure to free up memory
        plt.close(fig)

        # Save the PDF file to disk
        pdf_pages.close()


In [65]:
#
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.dates as mdates

# Assuming df is your DataFrame and 'Time' is your datetime column
df_temp = df.copy()
df_temp['Time'] = pd.to_datetime(df_temp['Time'])  # Convert 'Time' to datetime
df_temp.set_index('Time', inplace=True)  # Set 'Time' as index

# Add new columns for Year and Day of Week
df_temp['Year'] = df_temp.index.year
df_temp['Day_of_Week'] = df_temp.index.dayofweek

# Get numeric columns
numeric_columns = df_temp.select_dtypes(include=[np.number]).columns.tolist()

# Get unique years
years = df_temp['Year'].unique().tolist()
years.append('ALL')

output_folder = folder_path  # Specify your output folder here

for year in years:
    for column in numeric_columns:
        # Create a PDF object with specific file name

        if not os.path.exists(os.path.join(output_folder,'Yearly_Trend')):
            os.mkdir(os.path.join(output_folder,'Yearly_Trend'))


        pdf_pages = PdfPages(f'{output_folder}/Yearly_Trend/Yearly_Trend_{column}_{year}.pdf')

        if year == 'ALL':
            data = df_temp.copy()
        else:
            data = df_temp[df_temp['Year'] == year]

        monthly_avg = data[column].resample('M').mean()

        fig, ax = plt.subplots(figsize=(10, 6))
        ax.plot(monthly_avg.index, monthly_avg, label='Yearly Trend of {} in {}'.format(column, year))
        ax.set_title(f'Yearly Trend of {column} in {year}')
        ax.set_xlabel('Time')
        ax.set_ylabel(column)
        ax.legend()

        # Save the figure to the PDF file
        pdf_pages.savefig(fig)

        # Close the figure to free up memory
        plt.close(fig)

        # Save the PDF file to disk
        pdf_pages.close()


In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns

# Assuming df is your DataFrame
df_temp = df.copy()

# Select numeric columns
numeric_columns = df_temp.select_dtypes(include=[np.number]).columns.tolist()

# Select groupable object columns with no more than 15 unique values
group_columns = [col for col in df_temp.select_dtypes(include=['object']).columns if df_temp[col].nunique() <= 15]

output_folder = folder_path  # Specify your output folder here

for num_col in numeric_columns:
    for group_col in group_columns:
        # Create a box plot for each group in the group column
        plt.figure(figsize=(10, 6))
        sns.boxplot(x=group_col, y=num_col, data=df_temp)

        plt.title(f'Box Plot of {num_col} for Different {group_col} Groups')

        if not os.path.exists(os.path.join(output_folder,'Box_Plot_for_Different_Groups')):
            os.mkdir(os.path.join(output_folder,'Box_Plot_for_Different_Groups'))

        pdf_pages = PdfPages(f'{output_folder}/Box_Plot_for_Different_Groups/Box_Plot_{num_col}_for_{group_col}_Groups.pdf')
        pdf_pages.savefig(plt.gcf())
        plt.close()
        pdf_pages.close()



In [71]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.dates as mdates

# Assuming df is your DataFrame and 'Time' is your datetime index
df_temp = df.copy()
df_temp['Time'] = pd.to_datetime(df_temp[Time])  # Convert 'Time' to datetime
df_temp.set_index('Time', inplace=True)  # Set 'Time' as index

numeric_columns = df_temp.select_dtypes(include=[np.number]).columns.tolist()  # Get numeric columns

output_folder = folder_path   # Specify your output folder here

for column in numeric_columns:
    monthly_avg = df_temp[column].resample('M').mean()

    plt.figure(figsize=(30, 15))
    plt.plot(monthly_avg.index, monthly_avg, label='Monthly Trend of {}'.format(column))
    
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
    
    plt.title(f'Monthly Trend of {column}', size=15)
    plt.xlabel('Time', size=12)
    plt.ylabel(column, size=12)
    plt.legend()

    if not os.path.exists(os.path.join(output_folder,'Monthly_Trend')):
        os.mkdir(os.path.join(output_folder,'Monthly_Trend'))

    pdf_pages = PdfPages(f'{output_folder}/Monthly_Trend/Monthly_Trend_of_{column}.pdf')
    pdf_pages.savefig(plt.gcf())
    plt.close()
    pdf_pages.close()


In [73]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Assuming df is your DataFrame and 'Time' is your datetime column
df_temp = df.copy()
df_temp['Time'] = pd.to_datetime(df_temp[Time])  # Convert 'Time' to datetime
df_temp.set_index('Time', inplace=True)  # Set 'Time' as index
df_temp['Day_of_Week'] = df_temp.index.dayofweek  # Add a new column for day of the week

numeric_columns = df_temp.select_dtypes(include=[np.number]).columns.tolist()  # Get numeric columns

output_folder = folder_path   # Specify your output folder here

for column in numeric_columns:
    plt.figure(figsize=(10, 6))
    
    # Create a box plot for each day of the week
    for day in range(7):  # Assuming Monday=0, Sunday=6
        day_data = df_temp[df_temp['Day_of_Week'] == day][column]
        plt.boxplot(day_data, positions=[day])

    plt.title(f'Box Plot of {column} for Each Day of the Week', size=15)
    plt.xlabel('Day of the Week', size=12)
    plt.ylabel(column, size=12)
    plt.xticks(range(7), ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

    if not os.path.exists(os.path.join(output_folder,'Box_Plot_Day_of_Week')):
        os.mkdir(os.path.join(output_folder,'Box_Plot_Day_of_Week'))

    pdf_pages = PdfPages(f'{output_folder}/Box_Plot_Day_of_Week/Box_Plot_{column}_Day_of_Week.pdf')
    pdf_pages.savefig(plt.gcf())
    plt.close()
    pdf_pages.close()


In [74]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Assuming df is your DataFrame and 'Time' is your datetime column
df_temp = df.copy()
df_temp['Time'] = pd.to_datetime(df_temp[Time])  # Convert 'Time' to datetime
df_temp.set_index('Time', inplace=True)  # Set 'Time' as index

numeric_columns = df_temp.select_dtypes(include=[np.number]).columns.tolist()  # Get numeric columns

output_folder = folder_path   # Specify your output folder here

for column in numeric_columns:
    yearly_avg = df_temp[column].resample('Y').mean()  # Calculate yearly average

    plt.figure(figsize=(10, 6))
    plt.bar(yearly_avg.index.year, yearly_avg, label='Yearly Average of {}'.format(column))

    plt.title(f'Yearly Average of {column}', size=15)
    plt.xlabel('Year', size=12)
    plt.ylabel('Average', size=12)
    plt.legend()

    if not os.path.exists(os.path.join(output_folder,'Yearly_Average')):
        os.mkdir(os.path.join(output_folder,'Yearly_Average'))

    pdf_pages = PdfPages(f'{output_folder}/Yearly_Average/Yearly_Average_{column}.pdf')
    pdf_pages.savefig(plt.gcf())
    plt.close()
    pdf_pages.close()


In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Assuming df is your DataFrame and 'Time' is your datetime column
df_temp = df.copy()
df_temp['Time'] = pd.to_datetime(df_temp[Time])  # Convert 'Time' to datetime
df_temp.set_index('Time', inplace=True)  # Set 'Time' as index

# Add a new column for Year
df_temp['Year'] = df_temp.index.year
df_temp['Quarter'] = df_temp.index.quarter

# Get numeric columns
numeric_columns = df_temp.select_dtypes(include=[np.number]).columns.tolist()

# Get unique years
years = df_temp['Year'].unique().tolist()
years.append('ALL')

output_folder = folder_path  # Specify your output folder here

for column in numeric_columns:
    for year in years:
        if year == 'ALL':
            df_year = df_temp
            filename_suffix = 'ALL'
        else:
            df_year = df_temp[df_temp['Year'] == year]
            filename_suffix = str(year)

        fig, axs = plt.subplots(1, 4, figsize=(24, 6), sharey=True)
        fig.suptitle(f'Box Plot of {column} for Each Quarter in {filename_suffix}')

        for quarter in range(1, 5):  # 1 to 4 quarters
            quarter_data = df_year[df_year['Quarter'] == quarter][column]
            axs[quarter - 1].boxplot(quarter_data.dropna())  # dropna() to remove NaN values
            axs[quarter - 1].set_title(f'Quarter {quarter}')

        if not os.path.exists(os.path.join(output_folder,'Boxplot_Quarter')):
            os.mkdir(os.path.join(output_folder,'Boxplot_Quarter'))

        pdf_pages = PdfPages(f'{output_folder}/Boxplot_Quarter/Boxplot_{column}_Quarter_{filename_suffix}.pdf')
        pdf_pages.savefig(fig, bbox_inches='tight')
        plt.close()
        pdf_pages.close()


In [88]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Assuming df is your DataFrame and 'Time' is your datetime column
df_temp = df.copy()
df_temp['Time'] = pd.to_datetime(df_temp[Time])  # Convert 'Time' to datetime
df_temp.set_index('Time', inplace=True)  # Set 'Time' as index

# Add a new column for Year and Day_of_Week
df_temp['Year'] = df_temp.index.year
df_temp['Day_of_Week'] = df_temp.index.dayofweek


# Get numeric columns
numeric_columns = df_temp.select_dtypes(include=[np.number]).columns.tolist()

# Get unique years
years = df_temp['Year'].unique().tolist()
years.append('ALL')

output_folder = folder_path   # Specify your output folder here

for column in numeric_columns:
    for year in years:
        if year == 'ALL':
            df_year = df_temp
            filename_suffix = 'ALL'
        else:
            df_year = df_temp[df_temp['Year'] == year]
            filename_suffix = str(year)
        
        weekday_avg = df_year.groupby('Day_of_Week')[column].mean()

        plt.figure(figsize=(10, 6))
        plt.plot(weekday_avg.index, weekday_avg, label='Average of {}'.format(column))

        plt.title(f'Average of {column} for Each Day of the Week in {filename_suffix}')
        plt.xlabel('Day of the Week')
        plt.ylabel('Average')
        plt.legend()

        if not os.path.exists(os.path.join(output_folder,'Average_Weekday')):
            os.mkdir(os.path.join(output_folder,'Average_Weekday'))

        pdf_pages = PdfPages(f'{output_folder}/Average_Weekday/Average_{column}_Weekday_{filename_suffix}.pdf')
        pdf_pages.savefig(plt.gcf(), bbox_inches='tight')
        plt.close()
        pdf_pages.close()


In [89]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Assuming df is your DataFrame and 'Time' is your datetime column
df_temp = df.copy()
df_temp['Time'] = pd.to_datetime(df_temp[Time])  # Convert 'Time' to datetime
df_temp.set_index('Time', inplace=True)  # Set 'Time' as index

# Add a new column for Year
df_temp['Year'] = df_temp.index.year

# Get numeric columns
numeric_columns = df_temp.select_dtypes(include=[np.number]).columns.tolist()

# Get unique years
years = df_temp['Year'].unique().tolist()
years.append('ALL')

output_folder = folder_path  # Specify your output folder here

for column in numeric_columns:
    for year in years:
        if year == 'ALL':
            df_year = df_temp
            filename_suffix = 'ALL'
        else:
            df_year = df_temp[df_temp['Year'] == year]
            filename_suffix = str(year)
        
        plt.figure(figsize=(10, 6))
        plt.boxplot(df_year[column].dropna(), vert=False)
        
        plt.title(f'Yearly Distribution of {column} in {filename_suffix}')
        plt.xlabel(column)
        
        if not os.path.exists(os.path.join(output_folder,'Yearly_Distribution')):
            os.mkdir(os.path.join(output_folder,'Yearly_Distribution'))

        pdf_pages = PdfPages(f'{output_folder}/Yearly_Distribution/Yearly_Distribution_{column}_{filename_suffix}.pdf')
        pdf_pages.savefig(plt.gcf(), bbox_inches='tight')
        plt.close()
        pdf_pages.close()


In [90]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Assuming df is your DataFrame and 'Time' is your datetime column
df_temp = df.copy()
df_temp['Time'] = pd.to_datetime(df_temp[Time])  # Convert 'Time' to datetime
df_temp.set_index('Time', inplace=True)  # Set 'Time' as index

# Add new columns for Year and Month
df_temp['Year'] = df_temp.index.year
df_temp['Month'] = df_temp.index.month

# Get numeric columns
numeric_columns = df_temp.select_dtypes(include=[np.number]).columns.tolist()

# Get unique years
years = df_temp['Year'].unique().tolist()
years.append('ALL')

output_folder = folder_path  # Specify your output folder here

for column in numeric_columns:
    for year in years:
        if year == 'ALL':
            df_year = df_temp
            filename_suffix = 'ALL'
        else:
            df_year = df_temp[df_temp['Year'] == year]
            filename_suffix = str(year)
        
        monthly_avg = df_year.groupby('Month')[column].mean()
        
        plt.figure(figsize=(10, 6))
        plt.bar(monthly_avg.index, monthly_avg.values)
        
        plt.title(f'Average {column} for Each Month in {filename_suffix}')
        plt.xlabel('Month')
        plt.ylabel('Average')

        if not os.path.exists(os.path.join(output_folder,'Monthly_Average')):
            os.mkdir(os.path.join(output_folder,'Monthly_Average'))
        
        pdf_pages = PdfPages(f'{output_folder}/Monthly_Average/Monthly_Average_{column}_{filename_suffix}.pdf')
        pdf_pages.savefig(plt.gcf(), bbox_inches='tight')
        plt.close()
        pdf_pages.close()


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Assuming df is your DataFrame and 'Time' is your datetime column
df_temp = df.copy()
df_temp['Time'] = pd.to_datetime(df_temp[Time])  # Convert 'Time' to datetime
df_temp.set_index('Time', inplace=True)  # Set 'Time' as index

# Add new columns for Year and Week
df_temp['Year'] = df_temp.index.year
df_temp['Week'] = df_temp.index.isocalendar().week


# Get numeric columns
numeric_columns = df_temp.select_dtypes(include=[np.number]).columns.tolist()

# Get unique years
years = df_temp['Year'].unique().tolist()
years.append('ALL')

output_folder = folder_path  # Specify your output folder here

for column in numeric_columns:
    for year in years:
        if year == 'ALL':
            df_year = df_temp
            filename_suffix = 'ALL'
        else:
            df_year = df_temp[df_temp['Year'] == year]
            filename_suffix = str(year)
        
        weeks = sorted(df_year['Week'].unique())
        fig, ax = plt.subplots(figsize=(30, 15))

        for week in weeks:
            week_data = df_year[df_year['Week'] == week][column]
            ax.boxplot(week_data, positions=[week])
        
        ax.set_title(f'Weekly Distribution of {column} in {filename_suffix}')
        ax.set_xlabel('Week')
        ax.set_ylabel('Distribution')

        if not os.path.exists(os.path.join(output_folder,'Weekly_Distribution')):
            os.mkdir(os.path.join(output_folder,'Weekly_Distribution'))

        pdf_pages = PdfPages(f'{output_folder}/Weekly_Distribution/Weekly_Distribution_{column}_{filename_suffix}.pdf')
        pdf_pages.savefig(fig, bbox_inches='tight')
        plt.close(fig)
        pdf_pages.close()


In [95]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Assuming df is your DataFrame and 'Time' is your datetime column
df_temp = df.copy()
df_temp['Time'] = pd.to_datetime(df_temp[Time])  # Convert 'Time' to datetime
df_temp.set_index('Time', inplace=True)  # Set 'Time' as index

# Add new columns for Year and Day
df_temp['Year'] = df_temp.index.year
df_temp['Day'] = df_temp.index.day

# Get numeric columns
numeric_columns = df_temp.select_dtypes(include=[np.number]).columns.tolist()

# Get unique years
years = df_temp['Year'].unique().tolist()
years.append('ALL')

output_folder = folder_path # Specify your output folder here

for column in numeric_columns:
    for year in years:
        if year == 'ALL':
            df_year = df_temp
            filename_suffix = 'ALL'
        else:
            df_year = df_temp[df_temp['Year'] == year]
            filename_suffix = str(year)
        
        days = sorted(df_year['Day'].unique())
        day_means = df_year.groupby('Day')[column].mean()
        
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.plot(days, day_means)
        
        ax.set_title(f'Trend in {column} Over Days of the Month in {filename_suffix}')
        ax.set_xlabel('Day of the Month')
        ax.set_ylabel('Average Value')

        if not os.path.exists(os.path.join(output_folder,'Day_of_Month_Trend')):
            os.mkdir(os.path.join(output_folder,'Day_of_Month_Trend'))

        pdf_pages = PdfPages(f'{output_folder}/Day_of_Month_Trend/Day_of_Month_Trend_{column}_{filename_suffix}.pdf')
        pdf_pages.savefig(fig, bbox_inches='tight')
        plt.close(fig)
        pdf_pages.close()


In [97]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Assuming df is your DataFrame and 'Time' is your datetime column
df_temp = df.copy()
df_temp['Time'] = pd.to_datetime(df_temp[Time])  # Convert 'Time' to datetime
df_temp.set_index('Time', inplace=True)  # Set 'Time' as index

# Add new columns for Year and Day
df_temp['Year'] = df_temp.index.year
df_temp['Day'] = df_temp.index.day

# Get numeric columns
numeric_columns = df_temp.select_dtypes(include=[np.number]).columns.tolist()

# Get unique years
years = df_temp['Year'].unique().tolist()
years.append('ALL')

output_folder = folder_path  # Specify your output folder here

for column in numeric_columns:
    for year in years:
        if year == 'ALL':
            df_year = df_temp
            filename_suffix = 'ALL'
        else:
            df_year = df_temp[df_temp['Year'] == year]
            filename_suffix = str(year)
        
        days = sorted(df_year['Day'].unique())
        
        fig, ax = plt.subplots(figsize=(10, 6))
        
        # Prepare data for bar plot
        plot_data = df_year.groupby('Day')[column].mean()
        
        ax.bar(days, plot_data)
        
        ax.set_title(f'Average {column} for Each Day of the Month in {filename_suffix}')
        ax.set_xlabel('Day of the Month')
        ax.set_ylabel(f'Average {column}')

        if not os.path.exists(os.path.join(output_folder,'Daily_Average')):
            os.mkdir(os.path.join(output_folder,'Daily_Average'))

        pdf_pages = PdfPages(f'{output_folder}/Daily_Average/Daily_Average_{column}_{filename_suffix}.pdf')
        pdf_pages.savefig(fig, bbox_inches='tight')
        plt.close(fig)
        pdf_pages.close()


In [98]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns

# Assuming df is your DataFrame and 'Time' is your datetime column
df_temp = df.copy()
df_temp['Time'] = pd.to_datetime(df_temp[Time])  # Convert 'Time' to datetime
df_temp.set_index('Time', inplace=True)  # Set 'Time' as index

# Add new columns for Year and Day
df_temp['Year'] = df_temp.index.year
df_temp['Day'] = df_temp.index.day

# Get numeric columns
numeric_columns = df_temp.select_dtypes(include=[np.number]).columns.tolist()

# Get unique years
years = df_temp['Year'].unique().tolist()
years.append('ALL')

output_folder = folder_path  # Specify your output folder here

for column in numeric_columns:
    for year in years:
        if year == 'ALL':
            df_year = df_temp
            filename_suffix = 'ALL'
        else:
            df_year = df_temp[df_temp['Year'] == year]
            filename_suffix = str(year)
        
        days = sorted(df_year['Day'].unique())
        
        fig, ax = plt.subplots(figsize=(10, 6))
        
        # Prepare data for violin plot
        plot_data = [df_year[df_year['Day'] == day][column].values for day in days]
        
        sns.violinplot(data=plot_data, ax=ax, palette='Set3')
        
        ax.set_title(f'Daily Distribution of {column} in {filename_suffix}')
        ax.set_xlabel('Day of the Month')
        ax.set_ylabel(f'{column} Values')

        if not os.path.exists(os.path.join(output_folder,'Daily_Distribution')):
            os.mkdir(os.path.join(output_folder,'Daily_Distribution'))

        pdf_pages = PdfPages(f'{output_folder}/Daily_Distribution/Daily_Distribution_{column}_{filename_suffix}.pdf')
        pdf_pages.savefig(fig, bbox_inches='tight')
        plt.close(fig)
        pdf_pages.close()


In [19]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np

df_temp = df.copy()
df_temp['Time'] = pd.to_datetime(df_temp[Time])
df_temp.set_index('Time', inplace=True)

df_temp['Year'] = df_temp.index.year
df_temp['Month'] = df_temp.index.month

numeric_columns = df_temp.select_dtypes(include=[np.number]).columns.tolist()

years = df_temp['Year'].unique().tolist()
years.append('ALL')

output_folder = folder_path

for column in numeric_columns:
    for year in years:
        if year == 'ALL':
            df_year = df_temp
            filename_suffix = 'ALL'
        else:
            df_year = df_temp[df_temp['Year'] == year]
            filename_suffix = str(year)
        
        plot_data = []
        plot_months = []  # To store months corresponding to data in plot_data
        for month in range(1, 13):  # Iterate over all possible months
            month_data = df_year[df_year['Month'] == month][column].dropna().values
            if month_data.size > 1:  # Ensure there are at least 2 data points
                plot_data.append(month_data)
                plot_months.append(month)  # Add month to plot_months

        if plot_data:  # If plot_data is not empty
            try:  # Add a try-except block to catch potential errors
                fig, ax = plt.subplots(figsize=(10, 6))

                ax.violinplot(plot_data, positions=plot_months, showmeans=True)
                ax.set_title(f'Monthly Distribution of {column} in {filename_suffix}')
                ax.set_xlabel('Month')
                ax.set_ylabel('Value')
                ax.set_xticks(range(1, 13))  # Set x-ticks to be the months

                if not os.path.exists(os.path.join(output_folder,'Monthly_Distribution')):
                    os.mkdir(os.path.join(output_folder,'Monthly_Distribution'))

                pdf_pages = PdfPages(f'{output_folder}/Monthly_Distribution/Monthly_Distribution_{column}_{filename_suffix}.pdf')
                pdf_pages.savefig(fig, bbox_inches='tight')
                pdf_pages.close()

                plt.close(fig)
            except Exception as e:
                print(f"Cannot plot for {column} in {filename_suffix}: {e}")
